# Notebook for easily load and compare Numpy arrays created with TubeMap in Napari 

This notebook allows to compare the different results that we got from tubemap postproccessing steps to determine the best number of rounds

Napari is a powerful tool that allows the visualization of n-dimensional data easily in Python.

Even if ClearMap comes with a many tools to visualize the data (Visualization). We have noted that the capability of rendering 3D structures in Napari helps to analize the data during the developing of the pipeline.

First, lets initialize ClearMap to analize the images that we want to compare.

In [1]:
#ClearMap path
import sys
sys.path.append('/home/alain/Programs/ClearMap2/ClearMap2/')

#load ClearMap modules
from ClearMap.Environment import *  #analysis:ignore

/home/alain/anaconda3/envs/ClearMap3/lib/python3.7/site-packages/vispy/visuals/line/line.py:395: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  ('color', np.float32, 4)])
/home/alain/anaconda3/envs/ClearMap3/lib/python3.7/site-packages/vispy/visuals/line/arrow.py:57: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  ('linewidth', np.float32, 1)
/home/alain/anaconda3/envs/ClearMap3/lib/python3.7/site-packages/vispy/visuals/isocurve.py:22: UserWarning: VisPy is not yet compatible with matplotlib 2.2+
  warnings.warn("VisPy is not yet compatible with matplotlib 2.2+")


Elastix sucessfully initialized from path: /home/alain/Programs/ClearMap2/ClearMap2/ClearMap/External/elastix/build


Initialize workspace
The following set up the directories and filenames for a TubeMap project.

The raw files and files generated during the analysis of a data set are managed via the workspace.

It is a good practice to create a single folder for each skeletonization and keep your source files in a separate folder under your working directory.

Also, remember that some libraries don't handle spaces in the filenames, so avoid them.

In [2]:
#directories and files
directory = '/media/alain/DataWS6/Elena/20220120'  

expression_raw      = 'Tom-3R-P10-40x_Binary Mask.aivia.npy'
#expression_binary   = '/EH3603_Testis_PCW7_4X_1X_4ahCD31_5rbSMA_6msPLVAP_7gtSOX9_2u_LR_2_PLVAP.aivia.tif'

resources_directory = settings.resources_path

ws = wsp.Workspace('TubeMap', directory=directory);
ws.update(raw=expression_raw)
ws.info()

Workspace[TubeMap]{/media/alain/DataWS6/Elena/20220120}
              raw: no file
 autofluorescence: no file
         stitched: no file
           layout: no file
       background: no file
        resampled: no file
resampled_to_auto: no file
auto_to_reference: no file
         arteries: no file
           binary: no file
    binary_status: no file
         skeleton: no file
            graph: no file
          density: no file



First we have to convert our binary generated mask in Aivia to a numpy array so we can work with it in TubeMap

In [3]:
#Convert raw data to npy files     
               
io.convert_files(ws.file_list('raw', extension='tif'), extension='npy',
                 processes=70, verbose=True);

Converting 1 files to npy!
Converting file 0/1 Tif-Source(1026, 1934, 78)[uint8]{/media/alain/DataWS6/Elena/20220120/Tom-3R-P10-40x_Binary Mask.aivia.tif} -> /media/alain/DataWS6/Elena/20220120/Tom-3R-P10-40x_Binary Mask.aivia.npy
Converting 1 files to npy: elapsed time: 0:00:01.875


As the Mask generated in Aivia has the postive pixel in the Max 8-bit intensity (255), we have to covert it to a 0-1 scale using TubeMap

In [4]:
source = ws.filename('raw')
sink   = ws.filename('binary', postfix='TubeMap');


shape = io.shape(source)

#vesselized = ws.create('binary', postfix='vesselized', shape=shape, dtype=float)
#equalize = ws.create('binary', postfix='equalize', shape=shape, dtype=float)
#status = ws.create('binary', postfix='status', shape=shape, dtype='uint16')
#deconvolved = ws.create('binary', postfix='deconvolved', shape=shape, dtype=float)

binarization_parameter = vasc.default_binarization_parameter.copy();
binarization_parameter['clip']['clip_range'] = (1, 255)
binarization_parameter['lightsheet'] = None
binarization_parameter['median'] = None
#binarization_parameter['vesselize']['save'] = vesselized
binarization_parameter['adaptive'] = None
binarization_parameter['vesselize'] = None #480
binarization_parameter['deconvolve'] = None
binarization_parameter['equalize']['threshold'] = None #0.3 #2
#binarization_parameter['equalize']['save'] = equalize
#binarization_parameter['deconvolve']['save'] = deconvolved
#binarization_parameter['binary_status'] = status


processing_parameter = vasc.default_binarization_processing_parameter.copy();
processing_parameter['processes'] = 70;
processing_parameter['as_memory'] = None;
processing_parameter['verbose'] = True;

vasc.binarize(source, sink, binarization_parameter=binarization_parameter, processing_parameter=processing_parameter);

Processing 2 blocks with function 'binarize_block'.
Processing block 0/2<(0, 0, 0)/(1, 1, 2)> (1026, 1934, 39)@(1026, 1934, 78)[(:,:,0:39)]Processing block 1/2<(0, 0, 1)/(1, 1, 2)> (1026, 1934, 39)@(1026, 1934, 78)[(:,:,39:78)]

Block 0/2<(0, 0, 0)/(1, 1, 2)> (1026, 1934, 39)@(1026, 1934, 78)[(:,:,0:39)]: Clipping: clip_range: (1, 255)
Block 0/2<(0, 0, 0)/(1, 1, 2)> (1026, 1934, 39)@(1026, 1934, 78)[(:,:,0:39)]: Clipping: save      : FalseBlock 1/2<(0, 0, 1)/(1, 1, 2)> (1026, 1934, 39)@(1026, 1934, 78)[(:,:,39:78)]: Clipping: clip_range: (1, 255)
Block 1/2<(0, 0, 1)/(1, 1, 2)> (1026, 1934, 39)@(1026, 1934, 78)[(:,:,39:78)]: Clipping: save      : False

Block 0/2<(0, 0, 0)/(1, 1, 2)> (1026, 1934, 39)@(1026, 1934, 78)[(:,:,0:39)]: Clipping: elapsed time: 0:00:02.242
Block 0/2<(0, 0, 0)/(1, 1, 2)> (1026, 1934, 39)@(1026, 1934, 78)[(:,:,0:39)]: Equalization: percentile : (0.4, 0.975)
Block 0/2<(0, 0, 0)/(1, 1, 2)> (1026, 1934, 39)@(1026, 1934, 78)[(:,:,0:39)]: Equalization: selem      : (2

Now we can start to run the different smoothing rounds

In [5]:
  #Smoothing and filling 1
  
  source = ws.filename('binary', postfix='TubeMap');
  sink   = ws.filename('binary', postfix='postprocessed_1');
  
  postprocessing_parameter = vasc.default_postprocessing_parameter.copy();
  #postprocessing_parameter['fill'] = None;
  
  postprocessing_processing_parameter = vasc.default_postprocessing_processing_parameter.copy();
  postprocessing_processing_parameter.update(size_max=100);
  
  vasc.postprocess(source, sink, 
                   postprocessing_parameter=postprocessing_parameter, 
                   processing_parameter=postprocessing_processing_parameter, 
                   processes=70, verbose=True) 

Binary post processing: initialized.
Binary smoothing: initialized!
Processing 1 blocks with function 'smooth_by_configuration'.
Processing block 0/1<(0, 0, 0)/(1, 1, 1)> (1026, 1934, 78)@(1026, 1934, 78)[(:,:,0:78)]
Processing block 0/1<(0, 0, 0)/(1, 1, 1)> (1026, 1934, 78)@(1026, 1934, 78)[(:,:,0:78)]: elapsed time: 0:00:22.008
Processed 1 blocks with function 'smooth_by_configuration': elapsed time: 0:00:22.548
Binary smoothing: done: elapsed time: 0:00:22.549
Binary filling: initialized!
Binary filling: elapsed time: 0:00:00.411
Binary post processing: elapsed time: 0:00:23.040


In [6]:
  #Smoothing and filling 2
  
  source = ws.filename('binary', postfix='postprocessed_1');
  sink   = ws.filename('binary', postfix='postprocessed_2');
  
  postprocessing_parameter = vasc.default_postprocessing_parameter.copy();
  #postprocessing_parameter['fill'] = None;
  
  postprocessing_processing_parameter = vasc.default_postprocessing_processing_parameter.copy();
  postprocessing_processing_parameter.update(size_max=100);
  
  vasc.postprocess(source, sink, 
                   postprocessing_parameter=postprocessing_parameter, 
                   processing_parameter=postprocessing_processing_parameter, 
                   processes=70, verbose=True)

Binary post processing: initialized.
Binary smoothing: initialized!
Processing 1 blocks with function 'smooth_by_configuration'.
Processing block 0/1<(0, 0, 0)/(1, 1, 1)> (1026, 1934, 78)@(1026, 1934, 78)[(:,:,0:78)]
Processing block 0/1<(0, 0, 0)/(1, 1, 1)> (1026, 1934, 78)@(1026, 1934, 78)[(:,:,0:78)]: elapsed time: 0:00:21.913
Processed 1 blocks with function 'smooth_by_configuration': elapsed time: 0:00:22.525
Binary smoothing: done: elapsed time: 0:00:22.526
Binary filling: initialized!
Binary filling: elapsed time: 0:00:00.349
Binary post processing: elapsed time: 0:00:22.987


In [7]:
  # Smoothing and filling 3
  
  source = ws.filename('binary', postfix='postprocessed_2');
  sink   = ws.filename('binary', postfix='postprocessed_3');
  
  postprocessing_parameter = vasc.default_postprocessing_parameter.copy();
  #postprocessing_parameter['fill'] = None;
  
  postprocessing_processing_parameter = vasc.default_postprocessing_processing_parameter.copy();
  postprocessing_processing_parameter.update(size_max=100);
  
  vasc.postprocess(source, sink, 
                   postprocessing_parameter=postprocessing_parameter, 
                   processing_parameter=postprocessing_processing_parameter, 
                   processes=70, verbose=True)

Binary post processing: initialized.
Binary smoothing: initialized!
Processing 1 blocks with function 'smooth_by_configuration'.
Processing block 0/1<(0, 0, 0)/(1, 1, 1)> (1026, 1934, 78)@(1026, 1934, 78)[(:,:,0:78)]
Processing block 0/1<(0, 0, 0)/(1, 1, 1)> (1026, 1934, 78)@(1026, 1934, 78)[(:,:,0:78)]: elapsed time: 0:00:22.014
Processed 1 blocks with function 'smooth_by_configuration': elapsed time: 0:00:22.628
Binary smoothing: done: elapsed time: 0:00:22.629
Binary filling: initialized!
Binary filling: elapsed time: 0:00:00.359
Binary post processing: elapsed time: 0:00:23.082


In [ ]:
  # Smoothing and filling 4
  
  source = ws.filename('binary', postfix='postprocessed_200_3');
  sink   = ws.filename('binary', postfix='postprocessed_200_4');
  
  postprocessing_parameter = vasc.default_postprocessing_parameter.copy();
  #postprocessing_parameter['fill'] = None;
  
  postprocessing_processing_parameter = vasc.default_postprocessing_processing_parameter.copy();
  postprocessing_processing_parameter.update(size_max=100);
  
  vasc.postprocess(source, sink, 
                   postprocessing_parameter=postprocessing_parameter, 
                   processing_parameter=postprocessing_processing_parameter, 
                   processes=70, verbose=True)

## Result comparison

Now lets compare the different smoothing results with the mask generated with Aivia and the Raw data

First, lets run the magic code to activate the interface

In [ ]:
%gui qt

Now that we've created our GUI app, we can import napari and display an empty viewer

In [ ]:
import napari
from tifffile import imread
viewer = napari.Viewer()

Now we can import our Numpy arrays or images directly on Napari adding new layers to the viewer

In [ ]:
raw = np.load (ws.filename('raw'))
mask = np.load(ws.filename('binary', postfix='TubeMap'))
post1 = np.load(ws.filename('binary', postfix='postprocessed_1'))
post2 = np.load(ws.filename('binary', postfix='postprocessed_2'))
post3 = np.load(ws.filename('binary', postfix='postprocessed_3'))
post4 = np.load(ws.filename('binary', postfix='postprocessed_4'))

viewer.add_image(raw)
viewer.add_image(mask)
viewer.add_image(post1)
viewer.add_image(post2)
viewer.add_image(post3)
viewer.add_image(post4)

## Skeleton Construction

Now we are going to construct the different skeletons in order to compare them

In [8]:
# Skeletonize
  
binary   = ws.filename('binary', postfix='TubeMap');
skeleton = ws.filename('skeleton', postfix='binary')                   
  
skl.skeletonize(binary, sink=skeleton, delete_border=True, verbose=True);

#############################################################
Skeletonization PK12 [convolution, index]
Foreground points: 765875: elapsed time: 0:00:00.035
#############################################################
Iteration 1
<class 'numpy.ndarray'> int64 bool
uint8 int64 uint8 int64 int64 uint8
Border points: 344978: elapsed time: 0:00:00.006
-------------------------------------------------------------
Sub-Iteration 0
Matched points  : 88797: elapsed time: 0:00:00.006
Sub-Iteration 0: elapsed time: 0:00:00.009
-------------------------------------------------------------
Sub-Iteration 1
Matched points  : 35804: elapsed time: 0:00:00.009
Sub-Iteration 1: elapsed time: 0:00:00.011
-------------------------------------------------------------
Sub-Iteration 2
Matched points  : 80279: elapsed time: 0:00:00.005
Sub-Iteration 2: elapsed time: 0:00:00.007
-------------------------------------------------------------
Sub-Iteration 3
Matched points  : 53498: elapsed time: 0:00:00.005
Sub-

Matched points  : 609: elapsed time: 0:00:00.000
Sub-Iteration 7: elapsed time: 0:00:00.001
-------------------------------------------------------------
Sub-Iteration 8
Matched points  : 782: elapsed time: 0:00:00.014
Sub-Iteration 8: elapsed time: 0:00:00.014
-------------------------------------------------------------
Sub-Iteration 9
Matched points  : 1002: elapsed time: 0:00:00.005
Sub-Iteration 9: elapsed time: 0:00:00.006
-------------------------------------------------------------
Sub-Iteration 10
Matched points  : 807: elapsed time: 0:00:00.000
Sub-Iteration 10: elapsed time: 0:00:00.000
-------------------------------------------------------------
Sub-Iteration 11
Matched points  : 771: elapsed time: 0:00:00.000
Sub-Iteration 11: elapsed time: 0:00:00.000
-------------------------------------------------------------
Non-removable points: 1608
Foreground points   : 22677
-------------------------------------------------------------
Iteration 9: elapsed time: 0:00:00.034
#####

Matched points  : 5: elapsed time: 0:00:00.000
Sub-Iteration 0: elapsed time: 0:00:00.001
-------------------------------------------------------------
Sub-Iteration 1
Matched points  : 21: elapsed time: 0:00:00.002
Sub-Iteration 1: elapsed time: 0:00:00.003
-------------------------------------------------------------
Sub-Iteration 2
Matched points  : 8: elapsed time: 0:00:00.022
Sub-Iteration 2: elapsed time: 0:00:00.022
-------------------------------------------------------------
Sub-Iteration 3
Matched points  : 42: elapsed time: 0:00:00.005
Sub-Iteration 3: elapsed time: 0:00:00.006
-------------------------------------------------------------
Sub-Iteration 4
Matched points  : 10: elapsed time: 0:00:00.000
Sub-Iteration 4: elapsed time: 0:00:00.000
-------------------------------------------------------------
Sub-Iteration 5
Matched points  : 22: elapsed time: 0:00:00.000
Sub-Iteration 5: elapsed time: 0:00:00.000
-------------------------------------------------------------
Sub-

In [9]:
# Skeletonize1
  
binary   = ws.filename('binary', postfix='postprocessed_1');
skeleton = ws.filename('skeleton', postfix='postprocessed_1')                   
  
skl.skeletonize(binary, sink=skeleton, delete_border=True, verbose=True);

#############################################################
Skeletonization PK12 [convolution, index]
Foreground points: 769241: elapsed time: 0:00:00.041
#############################################################
Iteration 1
<class 'numpy.ndarray'> int64 bool
uint8 int64 uint8 int64 int64 uint8
Border points: 320059: elapsed time: 0:00:00.006
-------------------------------------------------------------
Sub-Iteration 0
Matched points  : 97840: elapsed time: 0:00:00.004
Sub-Iteration 0: elapsed time: 0:00:00.006
-------------------------------------------------------------
Sub-Iteration 1
Matched points  : 39953: elapsed time: 0:00:00.004
Sub-Iteration 1: elapsed time: 0:00:00.006
-------------------------------------------------------------
Sub-Iteration 2
Matched points  : 87669: elapsed time: 0:00:00.004
Sub-Iteration 2: elapsed time: 0:00:00.006
-------------------------------------------------------------
Sub-Iteration 3
Matched points  : 55705: elapsed time: 0:00:00.003
Sub-

uint8 int64 uint8 int64 int64 uint8
Border points: 1515: elapsed time: 0:00:00.005
-------------------------------------------------------------
Sub-Iteration 0
Matched points  : 10: elapsed time: 0:00:00.000
Sub-Iteration 0: elapsed time: 0:00:00.000
-------------------------------------------------------------
Sub-Iteration 1
Matched points  : 39: elapsed time: 0:00:00.000
Sub-Iteration 1: elapsed time: 0:00:00.000
-------------------------------------------------------------
Sub-Iteration 2
Matched points  : 9: elapsed time: 0:00:00.000
Sub-Iteration 2: elapsed time: 0:00:00.000
-------------------------------------------------------------
Sub-Iteration 3
Matched points  : 53: elapsed time: 0:00:00.000
Sub-Iteration 3: elapsed time: 0:00:00.000
-------------------------------------------------------------
Sub-Iteration 4
Matched points  : 22: elapsed time: 0:00:00.000
Sub-Iteration 4: elapsed time: 0:00:00.000
-------------------------------------------------------------
Sub-Iterati

In [10]:
# Skeletonize2
  
binary   = ws.filename('binary', postfix='postprocessed_2');
skeleton = ws.filename('skeleton', postfix='postprocessed_2')                   
  
skl.skeletonize(binary, sink=skeleton, delete_border=True, verbose=True);

#############################################################
Skeletonization PK12 [convolution, index]
Foreground points: 766536: elapsed time: 0:00:00.067
#############################################################
Iteration 1
<class 'numpy.ndarray'> int64 bool
uint8 int64 uint8 int64 int64 uint8
Border points: 317546: elapsed time: 0:00:00.006
-------------------------------------------------------------
Sub-Iteration 0
Matched points  : 96839: elapsed time: 0:00:00.004
Sub-Iteration 0: elapsed time: 0:00:00.006
-------------------------------------------------------------
Sub-Iteration 1
Matched points  : 39625: elapsed time: 0:00:00.003
Sub-Iteration 1: elapsed time: 0:00:00.005
-------------------------------------------------------------
Sub-Iteration 2
Matched points  : 87167: elapsed time: 0:00:00.003
Sub-Iteration 2: elapsed time: 0:00:00.006
-------------------------------------------------------------
Sub-Iteration 3
Matched points  : 55291: elapsed time: 0:00:00.003
Sub-

Matched points  : 382: elapsed time: 0:00:00.000
Sub-Iteration 11: elapsed time: 0:00:00.001
-------------------------------------------------------------
Foreground points   : 5739
-------------------------------------------------------------
Iteration 10: elapsed time: 0:00:00.019
#############################################################
Iteration 11
<class 'numpy.ndarray'> int64 bool
uint8 int64 uint8 int64 int64 uint8
Border points: 5739: elapsed time: 0:00:00.002
-------------------------------------------------------------
Sub-Iteration 0
Matched points  : 46: elapsed time: 0:00:00.000
Sub-Iteration 0: elapsed time: 0:00:00.000
-------------------------------------------------------------
Sub-Iteration 1
Matched points  : 158: elapsed time: 0:00:00.000
Sub-Iteration 1: elapsed time: 0:00:00.000
-------------------------------------------------------------
Sub-Iteration 2
Matched points  : 59: elapsed time: 0:00:00.000
Sub-Iteration 2: elapsed time: 0:00:00.000
---------------

Foreground points   : 557
-------------------------------------------------------------
Iteration 26: elapsed time: 0:00:00.014
#############################################################
Skeletonization done: elapsed time: 0:00:00.604
Total removed:   747997
Total remaining: 18539
Skeletonization: elapsed time: 0:00:00.663


In [11]:
# Skeletonize3
  
binary   = ws.filename('binary', postfix='postprocessed_3');
skeleton = ws.filename('skeleton', postfix='postprocessed_3')                   
  
skl.skeletonize(binary, sink=skeleton, delete_border=True, verbose=True);

#############################################################
Skeletonization PK12 [convolution, index]
Foreground points: 765136: elapsed time: 0:00:00.133
#############################################################
Iteration 1
<class 'numpy.ndarray'> int64 bool
uint8 int64 uint8 int64 int64 uint8
Border points: 316244: elapsed time: 0:00:00.006
-------------------------------------------------------------
Sub-Iteration 0
Matched points  : 96297: elapsed time: 0:00:00.004
Sub-Iteration 0: elapsed time: 0:00:00.006
-------------------------------------------------------------
Sub-Iteration 1
Matched points  : 39445: elapsed time: 0:00:00.003
Sub-Iteration 1: elapsed time: 0:00:00.005
-------------------------------------------------------------
Sub-Iteration 2
Matched points  : 86928: elapsed time: 0:00:00.004
Sub-Iteration 2: elapsed time: 0:00:00.006
-------------------------------------------------------------
Sub-Iteration 3
Matched points  : 55120: elapsed time: 0:00:00.003
Sub-

Matched points  : 3212: elapsed time: 0:00:00.000
Sub-Iteration 11: elapsed time: 0:00:00.001
-------------------------------------------------------------
Foreground points   : 15655
-------------------------------------------------------------
Iteration 7: elapsed time: 0:00:00.016
#############################################################
Iteration 8
<class 'numpy.ndarray'> int64 bool
uint8 int64 uint8 int64 int64 uint8
Border points: 14918: elapsed time: 0:00:00.001
-------------------------------------------------------------
Sub-Iteration 0
Matched points  : 803: elapsed time: 0:00:00.000
Sub-Iteration 0: elapsed time: 0:00:00.000
-------------------------------------------------------------
Sub-Iteration 1
Matched points  : 657: elapsed time: 0:00:00.000
Sub-Iteration 1: elapsed time: 0:00:00.000
-------------------------------------------------------------
Sub-Iteration 2
Matched points  : 526: elapsed time: 0:00:00.000
Sub-Iteration 2: elapsed time: 0:00:00.000
------------

Sub-Iteration 5: elapsed time: 0:00:00.001
-------------------------------------------------------------
Sub-Iteration 6
Matched points  : 25: elapsed time: 0:00:00.000
Sub-Iteration 6: elapsed time: 0:00:00.000
-------------------------------------------------------------
Sub-Iteration 7
Matched points  : 9: elapsed time: 0:00:00.001
Sub-Iteration 7: elapsed time: 0:00:00.002
-------------------------------------------------------------
Sub-Iteration 8
Matched points  : 18: elapsed time: 0:00:00.000
Sub-Iteration 8: elapsed time: 0:00:00.000
-------------------------------------------------------------
Sub-Iteration 9
Matched points  : 25: elapsed time: 0:00:00.000
Sub-Iteration 9: elapsed time: 0:00:00.000
-------------------------------------------------------------
Sub-Iteration 10
Matched points  : 19: elapsed time: 0:00:00.000
Sub-Iteration 10: elapsed time: 0:00:00.000
-------------------------------------------------------------
Sub-Iteration 11
Matched points  : 23: elapsed ti

In [ ]:
# Skeletonize4
  
binary   = ws.filename('binary', postfix='postprocessed_200_4');
skeleton = ws.filename('skeleton', postfix='postprocessed_200_4')                   
  
skl.skeletonize(binary, sink=skeleton, delete_border=True, verbose=True);

In [ ]:
# Skeletonize Filled
  
binary   = ws.filename('binary', postfix='filled');
skeleton = ws.filename('skeleton', postfix='filled')                   
  
skl.skeletonize(binary, sink=skeleton, delete_border=True, verbose=True);

In [ ]:
# Skeletonize Filled 2
  
binary   = ws.filename('binary', postfix='filled_2');
skeleton = ws.filename('skeleton', postfix='filled_2')                   
  
skl.skeletonize(binary, sink=skeleton, delete_border=True, verbose=True);

In [ ]:
# Skeletonize Filled
  
binary   = ws.filename('binary', postfix='filled_3');
skeleton = ws.filename('skeleton', postfix='filled_3')                   
  
skl.skeletonize(binary, sink=skeleton, delete_border=True, verbose=True);

In [ ]:
# Skeletonize Filled
  
binary   = ws.filename('binary', postfix='filled_TubeMap');
skeleton = ws.filename('skeleton', postfix='filled_TubeMap')                   
  
skl.skeletonize(binary, sink=skeleton, delete_border=True, verbose=True);

## Skeleton Comparison

Now we are going to open the ceated skeletons in Napari to compare them

In [ ]:
viewer = napari.Viewer()
skeleton_bin = np.load(ws.filename('skeleton', postfix='binary'))
skeleton_1 = np.load(ws.filename('skeleton', postfix='postprocessed_1'))
skeleton_2 = np.load(ws.filename('skeleton', postfix='postprocessed_2'))
skeleton_3 = np.load(ws.filename('skeleton', postfix='postprocessed_3'))
skeleton_4 = np.load(ws.filename('skeleton', postfix='postprocessed_4'))

viewer.add_image(mask)
viewer.add_image(post1)
viewer.add_image(post2)
viewer.add_image(post3)
viewer.add_image(post4)

viewer.add_image(skeleton_bin)
viewer.add_image(skeleton_1)
viewer.add_image(skeleton_2)
viewer.add_image(skeleton_3)
viewer.add_image(skeleton_4)


## Graph Construction

Now we are going to construct the graphs for the vascular tree using as base the skeletons that we got before

In [ ]:
# Graph from skeleton binary
  
graph_raw = gp.graph_from_skeleton(ws.filename('skeleton', postfix='binary'), verbose=True)
graph_raw.save(ws.filename('graph', postfix='binary'))

Now we will calculate the radius of the different vascular segments. for do this, we can use the binary reference or the raw data

In [ ]:
# Measure radii binary
  
coordinates = graph_raw.vertex_coordinates();   
radii, indices = mr.measure_radius(ws.filename('binary', postfix='TubeMap'), coordinates, 
                                     value=0, fraction=None, max_radius=300, 
  #                                   value=None, fraction=0.8, max_radius=150,
                                     return_indices=True, default=-1, verbose=True);  
graph_raw.set_vertex_radii(radii)

Now we will save the graph with the new radii for the vertex

In [ ]:
# Save raw graph binary

graph_raw.save(ws.filename('graph', postfix='binary'))

Then, some cleaning of unconnected segments and reduction of the graph is made for efficient management of the data

In [ ]:
# Graph binary cleaning

graph_cleaned = gp.clean_graph(graph_raw, 
                              vertex_mappings = {'coordinates'   : gp.mean_vertex_coordinates, 
                                                    'radii'         : np.max,
                                                    },                    
                                 verbose=True)  

In [ ]:
# Save cleaned graph binary
  
graph_cleaned.save(ws.filename('graph', postfix='cl_binary'))

In [ ]:
# Graph binary reduction
  
def vote(expression):
 return np.sum(expression) >= len(expression) / 1.5;
  
graph_reduced = gp.reduce_graph(graph_cleaned, edge_length=True,
                            edge_to_edge_mappings = {'length' : np.sum},
                            vertex_to_edge_mappings={'radii'  : np.max},  
                            edge_geometry_vertex_properties=['coordinates', 'radii'],
                            edge_geometry_edge_properties=None,                        
                            return_maps=False, verbose=True)

In [ ]:
# Save reduced graph binary

graph_reduced.save(ws.filename('graph', postfix='binary_reduced'))

Now we will run the same pipeline for the other smoothing binaries

In [ ]:
# Graph from skeleton postprocessed_1
  
graph_raw = gp.graph_from_skeleton(ws.filename('skeleton', postfix='postprocessed_1'), verbose=True)
graph_raw.save(ws.filename('graph', postfix='postprocessed_1'))

In [ ]:
# Measure radii postprocessed_1

coordinates = graph_raw.vertex_coordinates();   
radii, indices = mr.measure_radius(ws.filename('binary', postfix='postprocessed_1'), coordinates, 
                                     value=0, fraction=None, max_radius=300, 
  #                                   value=None, fraction=0.8, max_radius=150,
                                     return_indices=True, default=-1, verbose=True);  
graph_raw.set_vertex_radii(radii)

In [ ]:
# Save raw graph postprocessed_1
  
graph_raw.save(ws.filename('graph', postfix='postprocessed_1'))

In [ ]:
# Graph cleaning postprocessed_1
graph_cleaned = gp.clean_graph(graph_raw, 
                               vertex_mappings = {'coordinates'   : gp.mean_vertex_coordinates, 
                                                    'radii'         : np.max,
                                                    },                    
                               verbose=True)  

In [ ]:
# Save cleaned graph postprocessed_1
  
graph_cleaned.save(ws.filename('graph', postfix='cl_postprocessed_1'))

In [ ]:
# Graph reduction postprocessed_1
  
def vote(expression):
 return np.sum(expression) >= len(expression) / 1.5;
  
graph_reduced = gp.reduce_graph(graph_cleaned, edge_length=True,
                            edge_to_edge_mappings = {'length' : np.sum},
                            vertex_to_edge_mappings={'radii'  : np.max},  
                            edge_geometry_vertex_properties=['coordinates', 'radii'],
                            edge_geometry_edge_properties=None,                        
                            return_maps=False, verbose=True)

In [ ]:
# Save reduced graph postprocessed_1

graph_reduced.save(ws.filename('graph', postfix='postprocessed_1_reduced'))

In [ ]:
# Graph from skeleton postprocessed_2
  
graph_raw = gp.graph_from_skeleton(ws.filename('skeleton', postfix='postprocessed_2'), verbose=True)
graph_raw.save(ws.filename('graph', postfix='postprocessed_2'))

In [ ]:
# Measure radii postprocessed_2
  
coordinates = graph_raw.vertex_coordinates();   
radii, indices = mr.measure_radius(ws.filename('binary', postfix='postprocessed_2'), coordinates, 
                                     value=0, fraction=None, max_radius=300, 
  #                                   value=None, fraction=0.8, max_radius=150,
                                     return_indices=True, default=-1, verbose=True);  
graph_raw.set_vertex_radii(radii)

In [ ]:
# Save raw graph postprocessed_2
  
graph_raw.save(ws.filename('graph', postfix='postprocessed_2'))

In [ ]:
# Graph cleaning postprocessed_2
graph_cleaned = gp.clean_graph(graph_raw, 
                               vertex_mappings = {'coordinates'   : gp.mean_vertex_coordinates, 
                                                    'radii'         : np.max,
                                                    },                    
                               verbose=True)  

In [ ]:
# Save cleaned graph postprocessed_2
  
graph_cleaned.save(ws.filename('graph', postfix='cl_postprocessed_2'))

In [ ]:
# Graph reduction postprocessed_2
  
def vote(expression):
 return np.sum(expression) >= len(expression) / 1.5;
  
graph_reduced = gp.reduce_graph(graph_cleaned, edge_length=True,
                            edge_to_edge_mappings = {'length' : np.sum},
                            vertex_to_edge_mappings={'radii'  : np.max},  
                            edge_geometry_vertex_properties=['coordinates', 'radii'],
                            edge_geometry_edge_properties=None,                        
                            return_maps=False, verbose=True)

In [ ]:
# Save reduced graph postprocessed_2

graph_reduced.save(ws.filename('graph', postfix='postprocessed_2_reduced'))

In [ ]:
# Graph from skeleton postprocessed_3
  
graph_raw = gp.graph_from_skeleton(ws.filename('skeleton', postfix='postprocessed_3'), verbose=True)
graph_raw.save(ws.filename('graph', postfix='postprocessed_1'))

In [ ]:
# Measure radii postprocessed_3
  
coordinates = graph_raw.vertex_coordinates();   
radii, indices = mr.measure_radius(ws.filename('binary', postfix='postprocessed_3'), coordinates, 
                                     value=0, fraction=None, max_radius=300, 
  #                                   value=None, fraction=0.8, max_radius=150,
                                     return_indices=True, default=-1, verbose=True);  
graph_raw.set_vertex_radii(radii)

In [ ]:
# Save raw graph postprocessed_3
  
graph_raw.save(ws.filename('graph', postfix='postprocessed_3'))

In [ ]:
# Graph cleaning postprocessed_3
graph_cleaned = gp.clean_graph(graph_raw, 
                               vertex_mappings = {'coordinates'   : gp.mean_vertex_coordinates, 
                                                    'radii'         : np.max,
                                                    },                    
                               verbose=True)  

In [ ]:
# Save cleaned graph postprocessed_3
  
graph_cleaned.save(ws.filename('graph', postfix='cl_postprocessed_3'))

In [ ]:
# Graph reduction postprocessed_3
  
def vote(expression):
 return np.sum(expression) >= len(expression) / 1.5;
  
graph_reduced = gp.reduce_graph(graph_cleaned, edge_length=True,
                            edge_to_edge_mappings = {'length' : np.sum},
                            vertex_to_edge_mappings={'radii'  : np.max},  
                            edge_geometry_vertex_properties=['coordinates', 'radii'],
                            edge_geometry_edge_properties=None,                        
                            return_maps=False, verbose=True)

In [ ]:
# Save reduced graph postprocessed_3

graph_reduced.save(ws.filename('graph', postfix='postprocessed_3_reduced'))

In [ ]:
# Graph from skeleton postprocessed_4
  
graph_raw = gp.graph_from_skeleton(ws.filename('skeleton', postfix='postprocessed_4'), verbose=True)
graph_raw.save(ws.filename('graph', postfix='postprocessed_1'))

In [ ]:
# Measure radii postprocessed_4
  
coordinates = graph_raw.vertex_coordinates();   
radii, indices = mr.measure_radius(ws.filename('binary', postfix='postprocessed_4'), coordinates, 
                                     value=0, fraction=None, max_radius=300, 
  #                                   value=None, fraction=0.8, max_radius=150,
                                     return_indices=True, default=-1, verbose=True);  
graph_raw.set_vertex_radii(radii)

In [ ]:
# Save raw graph postprocessed_4
  
graph_raw.save(ws.filename('graph', postfix='postprocessed_4'))

In [ ]:
# Graph cleaning postprocessed_4
graph_cleaned = gp.clean_graph(graph_raw, 
                               vertex_mappings = {'coordinates'   : gp.mean_vertex_coordinates, 
                                                    'radii'         : np.max,
                                                    },                    
                               verbose=True)  

In [ ]:
# Save cleaned graph postprocessed_4
  
graph_cleaned.save(ws.filename('graph', postfix='cl_postprocessed_4'))

In [ ]:
# Graph reduction postprocessed_4
  
def vote(expression):
 return np.sum(expression) >= len(expression) / 1.5;
  
graph_reduced = gp.reduce_graph(graph_cleaned, edge_length=True,
                            edge_to_edge_mappings = {'length' : np.sum},
                            vertex_to_edge_mappings={'radii'  : np.max},  
                            edge_geometry_vertex_properties=['coordinates', 'radii'],
                            edge_geometry_edge_properties=None,                        
                            return_maps=False, verbose=True)

In [ ]:
# Save reduced graph postprocessed_4

graph_reduced.save(ws.filename('graph', postfix='postprocessed_4_reduced'))

## Plotting graphs

In [ ]:
graph_binary = grp.load(ws.filename('graph', postfix='binary_reduced'));
graph_P1 = grp.load(ws.filename('graph', postfix='postprocessed_1_reduced'));
graph_P2 = grp.load(ws.filename('graph', postfix='postprocessed_2_reduced'));
graph_P3 = grp.load(ws.filename('graph', postfix='postprocessed_3_reduced'));
graph_P4 = grp.load(ws.filename('graph', postfix='postprocessed_4_reduced'));

In [ ]:
radii = np.array(graph_binary.edge_property('radii'), dtype=int)
radii_colors = p3d.col.color_map('magma')(radii/radii.max());
p3d.plot_graph_line(graph_binary, edge_colors = radii_colors)

In [ ]:
import ClearMap.Analysis.Graphs.GraphRendering as gr

interpolation = gr.interpolate_edge_geometry(graph_binary, smooth=4, order=2, points_per_pixel=0.5, verbose=True);

coordinates, faces, colors = gr.mesh_tube_from_coordinates_and_radii(*interpolation,
                                    n_tube_points=10, edge_colors=radii_colors,
                                    processes=None, verbose=True);

p = p3d.plot_mesh_3d(coordinates, faces, vertex_colors=colors);

In [ ]:
radii = np.array(graph_P1.edge_property('radii'), dtype=int)
radii_colors = p3d.col.color_map('magma')(radii/radii.max());
p3d.plot_graph_line(graph_P1, edge_colors = radii_colors)

In [ ]:
import ClearMap.Analysis.Graphs.GraphRendering as gr

interpolation = gr.interpolate_edge_geometry(graph_P1, smooth=4, order=2, points_per_pixel=0.5, verbose=True);

coordinates, faces, colors = gr.mesh_tube_from_coordinates_and_radii(*interpolation,
                                    n_tube_points=10, edge_colors=radii_colors,
                                    processes=None, verbose=True);

p = p3d.plot_mesh_3d(coordinates, faces, vertex_colors=colors);

In [ ]:
radii = np.array(graph_P2.edge_property('radii'), dtype=int)
radii_colors = p3d.col.color_map('magma')(radii/radii.max());
p3d.plot_graph_line(graph_P2, edge_colors = radii_colors)

In [ ]:
import ClearMap.Analysis.Graphs.GraphRendering as gr

interpolation = gr.interpolate_edge_geometry(graph_P2, smooth=4, order=2, points_per_pixel=0.5, verbose=True);

coordinates, faces, colors = gr.mesh_tube_from_coordinates_and_radii(*interpolation,
                                    n_tube_points=10, edge_colors=radii_colors,
                                    processes=None, verbose=True);

p = p3d.plot_mesh_3d(coordinates, faces, vertex_colors=colors);

In [ ]:
radii = np.array(graph_P3.edge_property('radii'), dtype=int)
radii_colors = p3d.col.color_map('magma')(radii/radii.max());
p3d.plot_graph_line(graph_P3, edge_colors = radii_colors)

In [ ]:
import ClearMap.Analysis.Graphs.GraphRendering as gr

interpolation = gr.interpolate_edge_geometry(graph_P3, smooth=4, order=2, points_per_pixel=0.5, verbose=True);

coordinates, faces, colors = gr.mesh_tube_from_coordinates_and_radii(*interpolation,
                                    n_tube_points=10, edge_colors=radii_colors,
                                    processes=None, verbose=True);

p = p3d.plot_mesh_3d(coordinates, faces, vertex_colors=colors);

In [ ]:
radii = np.array(graph_P4.edge_property('radii'), dtype=int)
radii_colors = p3d.col.color_map('magma')(radii/radii.max());
p3d.plot_graph_line(graph_P4, edge_colors = radii_colors)

In [ ]:
import ClearMap.Analysis.Graphs.GraphRendering as gr

interpolation = gr.interpolate_edge_geometry(graph_P4, smooth=4, order=2, points_per_pixel=0.5, verbose=True);

coordinates, faces, colors = gr.mesh_tube_from_coordinates_and_radii(*interpolation,
                                    n_tube_points=10, edge_colors=radii_colors,
                                    processes=None, verbose=True);

p = p3d.plot_mesh_3d(coordinates, faces, vertex_colors=colors);

In [ ]:
source = ws.filename('binary', postfix='postprocessed_1');
sink   = ws.filename('binary', postfix='filled');
io.delete_file(sink)

processing_parameter = vf.default_fill_vessels_processing_parameter.copy();
processing_parameter.update(size_max = 200,
                            size_min = 'fixed',
                            axes = all,
                            overlap = 50);

vf.fill_vessels(source, sink,
                resample=1, threshold=0.5, cuda=True,
                processing_parameter=processing_parameter, verbose=True)

In [ ]:
source = ws.filename('binary', postfix='postprocessed_2');
sink   = ws.filename('binary', postfix='filled_2');
io.delete_file(sink)

processing_parameter = vf.default_fill_vessels_processing_parameter.copy();
processing_parameter.update(size_max = 200,
                            size_min = 'fixed',
                            axes = all,
                            overlap = 50);

vf.fill_vessels(source, sink,
                resample=1, threshold=0.5, cuda=True,
                processing_parameter=processing_parameter, verbose=True)

In [ ]:
source = ws.filename('binary', postfix='postprocessed_3');
sink   = ws.filename('binary', postfix='filled_3');
io.delete_file(sink)

processing_parameter = vf.default_fill_vessels_processing_parameter.copy();
processing_parameter.update(size_max = 200,
                            size_min = 'fixed',
                            axes = all,
                            overlap = 50);

vf.fill_vessels(source, sink,
                resample=1, threshold=0.5, cuda=True,
                processing_parameter=processing_parameter, verbose=True)

In [ ]:
source = ws.filename('binary', postfix='TubeMap');
sink   = ws.filename('binary', postfix='filled_TubeMap');
io.delete_file(sink)

processing_parameter = vf.default_fill_vessels_processing_parameter.copy();
processing_parameter.update(size_max = 200,
                            size_min = 'fixed',
                            axes = all,
                            overlap = 50);

vf.fill_vessels(source, sink,
                resample=1, threshold=0.5, cuda=True,
                processing_parameter=processing_parameter, verbose=True)

In [13]:
#Convert raw data to npy files     
               
io.convert_files(ws.file_list('skeleton', postfix='*', extension='npy'), extension='tif',
                 processes=70, verbose=True);

Converting 4 files to tif!
Converting file 0/4 Memmap-Source(1026, 1934, 78)[bool]|F|{/media/alain/DataWS6/Elena/20220120/skeleton_binary.npy} -> /media/alain/DataWS6/Elena/20220120/skeleton_binary.tif
Converting file 1/4 Memmap-Source(1026, 1934, 78)[bool]|F|{/media/alain/DataWS6/Elena/20220120/skeleton_postprocessed_1.npy} -> /media/alain/DataWS6/Elena/20220120/skeleton_postprocessed_1.tifConverting file 2/4 Memmap-Source(1026, 1934, 78)[bool]|F|{/media/alain/DataWS6/Elena/20220120/skeleton_postprocessed_2.npy} -> /media/alain/DataWS6/Elena/20220120/skeleton_postprocessed_2.tifConverting file 3/4 Memmap-Source(1026, 1934, 78)[bool]|F|{/media/alain/DataWS6/Elena/20220120/skeleton_postprocessed_3.npy} -> /media/alain/DataWS6/Elena/20220120/skeleton_postprocessed_3.tif


Converting 4 files to tif: elapsed time: 0:00:00.959
